In [1]:
import os
import numpy as np
import pandas as pd
df=pd.read_csv("dynamic_pricing")


FileNotFoundError: [Errno 2] No such file or directory: 'dynamic_pricing'

In [ ]:
df.info()

In [2]:
df=pd.read_csv("dynamic_pricing.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Number_of_Riders         1000 non-null   int64  
 1   Number_of_Drivers        1000 non-null   int64  
 2   Location_Category        1000 non-null   object 
 3   Customer_Loyalty_Status  1000 non-null   object 
 4   Number_of_Past_Rides     1000 non-null   int64  
 5   Average_Ratings          1000 non-null   float64
 6   Time_of_Booking          1000 non-null   object 
 7   Vehicle_Type             1000 non-null   object 
 8   Expected_Ride_Duration   1000 non-null   int64  
 9   Historical_Cost_of_Ride  1000 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 78.3+ KB


In [4]:
print("Shape:", df.shape)

Shape: (1000, 10)


In [5]:
print(df.describe())

       Number_of_Riders  Number_of_Drivers  Number_of_Past_Rides  \
count       1000.000000        1000.000000           1000.000000   
mean          60.372000          27.076000             50.031000   
std           23.701506          19.068346             29.313774   
min           20.000000           5.000000              0.000000   
25%           40.000000          11.000000             25.000000   
50%           60.000000          22.000000             51.000000   
75%           81.000000          38.000000             75.000000   
max          100.000000          89.000000            100.000000   

       Average_Ratings  Expected_Ride_Duration  Historical_Cost_of_Ride  
count      1000.000000              1000.00000              1000.000000  
mean          4.257220                99.58800               372.502623  
std           0.435781                49.16545               187.158756  
min           3.500000                10.00000                25.993449  
25%           3.8

In [6]:
print(df.describe(include=['object']))

       Location_Category Customer_Loyalty_Status Time_of_Booking Vehicle_Type
count               1000                    1000            1000         1000
unique                 3                       3               4            2
top                Urban                  Silver           Night      Premium
freq                 346                     367             276          522


In [7]:
print(df.isna().sum())

Number_of_Riders           0
Number_of_Drivers          0
Location_Category          0
Customer_Loyalty_Status    0
Number_of_Past_Rides       0
Average_Ratings            0
Time_of_Booking            0
Vehicle_Type               0
Expected_Ride_Duration     0
Historical_Cost_of_Ride    0
dtype: int64


In [8]:
print(df.duplicated().sum())

0


In [11]:
num_cols=df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols=df.select_dtypes(exclude=[np.number]).columns.tolist()
print("Categorical Columns:",cat_cols)

Categorical Columns: ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking', 'Vehicle_Type']


In [12]:
print("Numeric columns:",num_cols)

Numeric columns: ['Number_of_Riders', 'Number_of_Drivers', 'Number_of_Past_Rides', 'Average_Ratings', 'Expected_Ride_Duration', 'Historical_Cost_of_Ride']


In [4]:
import pandas as pd  

df = pd.read_csv("dynamic_pricing.csv")  
print(df.head())   # shows first 5 rows

   Number_of_Riders  Number_of_Drivers Location_Category  \
0                90                 45             Urban   
1                58                 39          Suburban   
2                42                 31             Rural   
3                89                 28             Rural   
4                78                 22             Rural   

  Customer_Loyalty_Status  Number_of_Past_Rides  Average_Ratings  \
0                  Silver                    13             4.47   
1                  Silver                    72             4.06   
2                  Silver                     0             3.99   
3                 Regular                    67             4.31   
4                 Regular                    74             3.77   

  Time_of_Booking Vehicle_Type  Expected_Ride_Duration  \
0           Night      Premium                      90   
1         Evening      Economy                      43   
2       Afternoon      Premium                      76  

In [5]:
cat_cols = df.select_dtypes(include=['object']).columns

# Unique values and counts
cat_uniques = {c: sorted(df[c].astype(str).unique().tolist()) for c in cat_cols}
cat_counts = {c: int(df[c].nunique()) for c in cat_cols}

print("Unique values per categorical column:")
for c in cat_cols:
    print(f"- {c} ({cat_counts[c]}): {cat_uniques[c]}")

Unique values per categorical column:
- Location_Category (3): ['Rural', 'Suburban', 'Urban']
- Customer_Loyalty_Status (3): ['Gold', 'Regular', 'Silver']
- Time_of_Booking (4): ['Afternoon', 'Evening', 'Morning', 'Night']
- Vehicle_Type (2): ['Economy', 'Premium']


In [6]:
inconsistency_flags = {}
for c in cat_cols:
    raw = df[c].astype(str)
    norm = raw.str.strip().str.lower()   # normalize (remove spaces + lowercase)
    inconsistency_flags[c] = int((raw != norm).sum())

print("\nPotential formatting inconsistencies (case/whitespace) per categorical col:")
print(inconsistency_flags)


Potential formatting inconsistencies (case/whitespace) per categorical col:
{'Location_Category': 1000, 'Customer_Loyalty_Status': 1000, 'Time_of_Booking': 1000, 'Vehicle_Type': 1000}


In [8]:
cat_cols = ["Location_Category", "Customer_Loyalty_Status", "Time_of_Booking", "Vehicle_Type"]

for col in cat_cols:
    # Remove extra spaces + lowercase for uniformity
    df[col] = df[col].str.strip().str.lower()
    
    # Replace multiple spaces with one
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

# ✅ Verify cleaned categories
for col in cat_cols:
    print(f"\n{col} unique values:\n", df[col].unique())



Location_Category unique values:
 ['urban' 'suburban' 'rural']

Customer_Loyalty_Status unique values:
 ['silver' 'regular' 'gold']

Time_of_Booking unique values:
 ['night' 'evening' 'afternoon' 'morning']

Vehicle_Type unique values:
 ['premium' 'economy']


In [9]:
def category_health_summary(df, cat_cols):
    summary = {}
    for col in cat_cols:
        counts = df[col].value_counts()
        percentages = df[col].value_counts(normalize=True) * 100
        summary[col] = pd.DataFrame({"Count": counts, "Percentage": percentages.round(2)})
    return summary

# Categorical columns
cat_cols = ["Location_Category", "Customer_Loyalty_Status", "Time_of_Booking", "Vehicle_Type"]

# Run the summary
summary = category_health_summary(df, cat_cols)

# Display results
for col, stats in summary.items():
    print(f"\n📊 {col} distribution:\n")
    print(stats)



📊 Location_Category distribution:

                   Count  Percentage
Location_Category                   
urban                346        34.6
rural                332        33.2
suburban             322        32.2

📊 Customer_Loyalty_Status distribution:

                         Count  Percentage
Customer_Loyalty_Status                   
silver                     367        36.7
regular                    320        32.0
gold                       313        31.3

📊 Time_of_Booking distribution:

                 Count  Percentage
Time_of_Booking                   
night              276        27.6
afternoon          247        24.7
morning            246        24.6
evening            231        23.1

📊 Vehicle_Type distribution:

              Count  Percentage
Vehicle_Type                   
premium         522        52.2
economy         478        47.8


In [10]:
import numpy as np

# Select only numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Dictionaries to store results
iqr_bounds = {}
iqr_outlier_counts = {}
z_outlier_counts = {}

# Outlier detection loop
for c in num_cols:
    # --- IQR Method ---
    q1, q3 = df[c].quantile(0.25), df[c].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    
    # Save IQR bounds
    iqr_bounds[c] = (round(lower, 2), round(upper, 2))
    
    # Mask outliers
    mask_iqr = (df[c] < lower) | (df[c] > upper)
    iqr_outlier_counts[c] = int(mask_iqr.sum())
    df[f"is_outlier_IQR_{c}"] = mask_iqr
    
    # --- Z-score Method ---
    mean, std = df[c].mean(), df[c].std(ddof=0)
    if std == 0:
        z_outlier_counts[c] = 0
        df[f"is_outlier_Z_{c}"] = False
    else:
        z = (df[c] - mean) / std
        mask_z = z.abs() > 3
        z_outlier_counts[c] = int(mask_z.sum())
        df[f"is_outlier_Z_{c}"] = mask_z

# Print results
print(" IQR bounds per numeric column:\n", iqr_bounds)
print("\n IQR outlier counts:\n", iqr_outlier_counts)
print("\n Z-score outlier counts:\n", z_outlier_counts)


 IQR bounds per numeric column:
 {'Number_of_Riders': (np.float64(-21.5), np.float64(142.5)), 'Number_of_Drivers': (np.float64(-29.5), np.float64(78.5)), 'Number_of_Past_Rides': (np.float64(-50.0), np.float64(150.0)), 'Average_Ratings': (np.float64(2.73), np.float64(5.78)), 'Expected_Ride_Duration': (np.float64(-65.12), np.float64(267.88)), 'Historical_Cost_of_Ride': (np.float64(-212.33), np.float64(944.2))}

 IQR outlier counts:
 {'Number_of_Riders': 0, 'Number_of_Drivers': 10, 'Number_of_Past_Rides': 0, 'Average_Ratings': 0, 'Expected_Ride_Duration': 0, 'Historical_Cost_of_Ride': 0}

 Z-score outlier counts:
 {'Number_of_Riders': 0, 'Number_of_Drivers': 4, 'Number_of_Past_Rides': 0, 'Average_Ratings': 0, 'Expected_Ride_Duration': 0, 'Historical_Cost_of_Ride': 0}


In [13]:
# Price (Recommended dynamic price) - assume 20% markup on cost if not present
if "Price" not in df.columns:
    df["Price"] = df["Historical_Cost_of_Ride"] * 1.2

# Baseline Price - assume 10% markup on cost
df["Baseline_Price"] = df["Historical_Cost_of_Ride"] * 1.1

# Completed Rides
if "Completed_Rides" not in df.columns:
    df["Completed_Rides"] = df["Number_of_Riders"]

# Synthetic Cancelled Rides (5–15% of completed rides, random)
if "Cancelled_Rides" not in df.columns:
    np.random.seed(42)
    df["Cancelled_Rides"] = (df["Completed_Rides"] *
                             np.random.uniform(0.05, 0.15, len(df))).astype(int)

# Booking Intents = Completed + Cancelled
df["Booking_Intents"] = df["Completed_Rides"] + df["Cancelled_Rides"]

# Cost per ride
df["Cost"] = df["Historical_Cost_of_Ride"]

# ----------------------------
# Step 2: KPI Calculations
# ----------------------------

# Revenue
df["Revenue"] = df["Price"] * df["Completed_Rides"]

# Baseline Revenue
df["Baseline_Revenue"] = df["Baseline_Price"] * df["Completed_Rides"]

# Profit
df["Profit"] = (df["Price"] - df["Cost"]) * df["Completed_Rides"]

# Revenue Lift (%)
df["Revenue_Lift_%"] = ((df["Revenue"] - df["Baseline_Revenue"]) /
                        df["Baseline_Revenue"]) * 100

# Gross Margin (%)
df["Gross_Margin_%"] = (df["Revenue"] - (df["Cost"] * df["Completed_Rides"])) / df["Revenue"] * 100

# Conversion Rate (%)
df["Conversion_Rate_%"] = (df["Completed_Rides"] / df["Booking_Intents"]) * 100

# Cancellation Rate (%)
df["Cancellation_Rate_%"] = (df["Cancelled_Rides"] / df["Booking_Intents"]) * 100

# Price Change Rate (%)
price_changes = (df["Price"].diff() != 0).sum()
Price_Change_Rate = (price_changes / len(df)) * 100

# ----------------------------
# Step 3: Aggregate KPI Summary
# ----------------------------
print("\n📊 KPI Summary (Aggregated)")
print(f"Total Revenue (₹): {df['Revenue'].sum():,.2f}")
print(f"Total Profit (₹): {df['Profit'].sum():,.2f}")
print(f"Revenue Lift (%): {df['Revenue_Lift_%'].mean():.2f}%")
print(f"Average Gross Margin (%): {df['Gross_Margin_%'].mean():.2f}%")
print(f"Average Conversion Rate (%): {df['Conversion_Rate_%'].mean():.2f}%")
print(f"Average Cancellation Rate (%): {df['Cancellation_Rate_%'].mean():.2f}%")
print(f"Price Change Rate (%): {Price_Change_Rate:.2f}%")

# ----------------------------
# Step 4: Ride-level KPI Dashboard
# ----------------------------
kpi_cols = [
    "Revenue", "Profit", "Revenue_Lift_%", "Gross_Margin_%",
    "Conversion_Rate_%", "Cancellation_Rate_%"
]
kpi_dashboard = df[kpi_cols]
kpi_dashboard.head(10)  # preview


📊 KPI Summary (Aggregated)
Total Revenue (₹): 27,017,454.02
Total Profit (₹): 4,502,909.00
Revenue Lift (%): 9.09%
Average Gross Margin (%): 16.67%
Average Conversion Rate (%): 91.86%
Average Cancellation Rate (%): 8.14%
Price Change Rate (%): 100.00%


,Revenue,Profit,Revenue_Lift_%,Gross_Margin_%,Conversion_Rate_%,Cancellation_Rate_%
0,30699.785486,5116.630914,9.090909,16.666667,92.783505,7.216495
1,12101.682791,2016.947132,9.090909,16.666667,87.878788,12.121212
2,16621.691636,2770.281939,9.090909,16.666667,89.361702,10.638298
3,50217.491551,8369.581925,9.090909,16.666667,90.816327,9.183673
4,54258.181133,9043.030189,9.090909,16.666667,93.975904,6.024096
5,24068.839531,4011.473255,9.090909,16.666667,95.161290,4.838710
6,11613.268006,1935.544668,9.090909,16.666667,94.897959,5.102041
7,17544.402649,2924.067108,9.090909,16.666667,88.571429,11.428571
8,47533.906657,7922.317776,9.090909,16.666667,90.804598,9.195402
9,20109.265574,3351.544262,9.090909,16.666667,89.361702,10.638298
